# Time Series Profiler

In [9]:
import os
import pandas as pd
import tsfel
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

ROOT = os.getcwd()
BUILDS = f"{ROOT}/../builds"
SCENARIOS = f"{ROOT}/../scenarios"

## Select a scenario
Set the SCENARIOS index value from the available scenarios

In [10]:
dir = []
def list_files(path="."):
    with os.scandir(path) as entries:
        for entry in entries:
            if entry.is_file():
                if "system.json" in entry.path:
                    path, _ = entry.path.split("system.json", 1)
                    dir.append(path)
            elif entry.is_dir():
                list_files(entry.path)

    

# Specify the directory path you want to start from
directory_path = f"{SCENARIOS}/lindistflow"
list_files(directory_path)
for i, d in enumerate(dir):
    print(i, d)

0 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/ieee123/
1 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/SFO/P9U/low/
2 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/SFO/P9U/high/
3 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/SFO/P9U/extreme/
4 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/SFO/P9U/medium/
5 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/SFO/P1U/low/
6 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/SFO/P1U/high/
7 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/SFO/P1U/extreme/
8 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/SFO/P1U/medium/
9 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/SFO/P6U/low/
10 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/SFO/P6U/high/
11 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lindistflow/SFO/P6U/extreme/
12 /home/tylor/dev/oedisi_dopf/profiler/../scenarios/lind

In [11]:
index = 0
path = dir[index]
_, scenario = path.split(SCENARIOS, 1)
build = f"{BUILDS}{scenario}"
profiles_dir = f"{build}feeder/profiles/load_profiles"
files = [e.name for e in os.scandir(profiles_dir) if e.is_file()]
profiles = [pd.read_csv(f"{profiles_dir}/{f}") for f in files]

In [12]:
# Step 2: Extract features from time-series using TSFEL
def extract_features(data):
    """Extract relevant features from time-series data using TSFEL."""
    # Load TSFEL feature configuration
    feature_extraction_settings = tsfel.get_features_by_domain()
    
    # Extract features
    features = tsfel.time_series_features_extractor(feature_extraction_settings, data, verbose=False)
    return features

feat = extract_features(profiles)

/tmp/ipykernel_371136/2771920099.py:8: UserWarning: Using default sampling frequency set in configuration file.
  features = tsfel.time_series_features_extractor(feature_extraction_settings, data, verbose=False)


In [13]:
## Normalize features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(feat.values)

In [15]:
print(scaled_features)

[[0.09962453 0.39817115 0.         ... 0.16096923 0.17098166 0.        ]
 [0.04438223 0.74061328 0.         ... 0.01915905 0.01348837 0.        ]
 [0.13207229 0.85670968 0.         ... 0.08826608 0.07556419 0.        ]
 ...
 [0.0553712  0.77800274 0.         ... 0.02521237 0.01662457 0.        ]
 [0.12403801 0.63428798 0.         ... 0.11934039 0.11973058 0.        ]
 [0.20064312 0.36586103 0.         ... 0.28011924 0.28959903 0.        ]]


In [14]:
# Step 3: Build GAN model
def build_gan(input_dim):
    """Build GAN model."""
    # Generator
    generator = models.Sequential([
        layers.Dense(128, activation="relu", input_dim=input_dim),
        layers.Dense(256, activation="relu"),
        layers.Dense(input_dim, activation="tanh")  # Output shape matches the input_dim
    ])
    
    # Discriminator
    discriminator = models.Sequential([
        layers.Dense(256, activation="relu", input_dim=input_dim),
        layers.Dense(128, activation="relu"),
        layers.Dense(1, activation="sigmoid")  # Binary classification output
    ])
    
    discriminator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])
    return generator, discriminator

generator, discriminator = build_gan(scaled_features)

/home/tylor/.cache/pypoetry/virtualenvs/profilier-6WcZE0tj-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Invalid dtype: ndarray

In [ ]:
# Step 4: Train GAN
def train_gan(generator, discriminator, data, epochs=1000, batch_size=32):
    """Train the GAN to create synthetic data."""
    # Get input dimension
    input_dim = data.shape[1]
    
    # Combine generator and discriminator to form GAN
    discriminator.trainable = False
    gan = models.Sequential([generator, discriminator])
    gan.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy")
    
    # Training loop
    for epoch in range(epochs):
        # ---------------------------
        # Train Discriminator
        # ---------------------------
        idx = np.random.randint(0, data.shape[0], batch_size)
        real_samples = data[idx]
        
        noise = np.random.normal(0, 1, (batch_size, input_dim))
        fake_samples = generator.predict(noise)
        
        labels_real = np.ones((batch_size, 1))
        labels_fake = np.zeros((batch_size, 1))
        
        d_loss_real = discriminator.train_on_batch(real_samples, labels_real)
        d_loss_fake = discriminator.train_on_batch(fake_samples, labels_fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # ---------------------------
        # Train Generator (via GAN)
        # ---------------------------
        noise = np.random.normal(0, 1, (batch_size, input_dim))
        labels_gan = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, labels_gan)
        
        # Print progress
        if epoch % 100 == 0 or epoch == epochs - 1:
            print(f"Epoch {epoch}/{epochs}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")
    
    return generator

trained_generator = train_gan(generator, discriminator, scaled_features)

In [ ]:
# Step 5: Generate synthetic data
def generate_synthetic_data(generator, n_samples=100):
    """Generate synthetic data using the trained generator."""
    noise = np.random.normal(0, 1, (n_samples, generator.input_shape[1]))
    synthetic_data = generator.predict(noise)
    return synthetic_data


# Generate synthetic features
synthetic_features = generate_synthetic_data(trained_generator)

In [ ]:
# Revert normalization to original scale if needed
synthetic_features_original_scale = scaler.inverse_transform(synthetic_features)

# Visualize results
plt.figure(figsize=(10, 6))
plt.plot(profiles[0], label="Original Time Series")
plt.plot(synthetic_features_original_scale[0], label="Synthetic Time Series")
plt.legend()
plt.title("Original vs. Synthetic Time Series")
plt.show()